In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from ipywidgets import interact

In [2]:
alfa = 5

In [3]:
def test_ez (x: np.ndarray) -> np.ndarray:
    return np.sum((x-alfa)**2)

In [4]:
def test_ez_grad (x: np.ndarray) -> np.ndarray:
    return (x-alfa)*2

In [5]:
def test_rust (x: np.ndarray): # Функция Растригина
    A = 10
    return A * x.size + np.sum(x ** 2 - A * np.cos(2*np.pi*x))

In [6]:
def test_styb (x: np.ndarray): # Функция Стыбинского-Танга
    return np.sum(x ** 4 - 16 * x ** 2 + 5 * x)/2

In [16]:
Acc = 500
x1 = np.linspace(-5, 5, Acc)
x2 = np.linspace(-5, 5, Acc)
Z = np.zeros((Acc, Acc))
Z2 = np.zeros((Acc, Acc))
for i in range (Acc):
    for j in range (Acc):
        Z[i, j] = test_rust(np.array([x1[i], x2[j]]))
        Z2[i, j] = test_styb(np.array([x1[i], x2[j]]))
        
def plot_3d_graph(angle_x, angle_y, alpha, x1, x2, Z):
    fig = plt.figure(figsize=(12, 6))  # Увеличьте ширину фигуры, чтобы разместить два графика рядом
    ax1 = fig.add_subplot(121, projection='3d')  # Создайте первую 3D-сетку
    ax2 = fig.add_subplot(122, projection='3d')  # Создайте вторую 3D-сетку

    X1, X2 = np.meshgrid(x1, x2)
    
    ax1.plot_surface(X1, X2, Z, alpha=alpha, cmap='viridis')
    ax2.plot_surface(X1, X2, Z2, alpha=alpha, cmap='viridis')
    
    ax1.view_init(elev=angle_x, azim=angle_y)
    ax2.view_init(elev=angle_x, azim=angle_y)

    plt.show()
graph = lambda angle_x, angle_y, alpha: plot_3d_graph(angle_x, angle_y, alpha, x1 = x1, x2 = x2, Z = Z)
# Создайте интерактивный виджет для прокрутки и вращения графика
interact(graph, angle_x=(-90, 90, 5), angle_y=(0, 360, 5), alpha=(0, 1, 0.1)) # Отрисовка Функции Растригина

interactive(children=(IntSlider(value=0, description='angle_x', max=90, min=-90, step=5), IntSlider(value=180,…

<function __main__.<lambda>(angle_x, angle_y, alpha)>

In [17]:
def num_diff(
    x: np.ndarray,
    use_function: bool = True, 
    h: float = 1e-7,
    **kwargs) -> np.ndarray:
    if use_function:
        f = kwargs['f']
    else:
        NotImplementedError()
    grad = np.zeros_like(x)
    
    for i in range(len(x)):
        original_value = x[i]
        
        # Вычисляем градиент по i-ой переменной с шагом h
        x[i] = original_value + h
        fx_plus_h = f(x)
        
        x[i] = original_value - h
        fx_minus_h = f(x)
        
        grad[i] = (fx_plus_h - fx_minus_h) / (2 * h)
        
        # Восстанавливаем исходное значение переменной x[i]
        x[i] = original_value
    
    return grad
    

In [18]:
def my_GD(
         x0: np.ndarray,
         T: int = 100,
         lr: float = 1e-3,
         grad_type: str = 'num_diff',
         use_crit: bool = False,
         crit: float = 1e-7,
         **kwargs) -> np.ndarray:    
    if grad_type != 'input' or grad_type != 'num_diff' or grad_type != 'sym_diff':
        grad_type = 'num_diff'
    if grad_type == 'input':
        df = kwargs['df']
    elif grad_type == 'num_diff':
        f = kwargs['f']
        df = lambda x: num_diff(x, f = f)
        
    x = x0
    if use_crit:
        while (np.sum(np.absolute(df(x))) > crit):
            x -= lr*df(x)
    else:
        for _ in range(T):
            x -= lr*df(x)
    return x

In [28]:
xt = np.random.normal(0, 1, 2)
my_GD(x0 = xt, use_crit = True, f = test_ez)

array([4.99999997, 4.99999998])

In [37]:
xt = np.random.normal(0, 1, 2)
my_GD(x0 = xt, use_crit = True, f = test_rust)

array([-9.94958637e-01, -1.17980625e-11])

In [38]:
xt =np.random.normal(0, 1, 2)
my_GD(x0 = xt, use_crit = True, f = test_styb)

array([-2.90353402, -2.90353402])

In [39]:
test_styb(xt)

-78.33233140754282

In [17]:
xt

array([ 1., -2.,  3.])